In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

Python-dotenv could not parse statement starting at line 16
Python-dotenv could not parse statement starting at line 17
Python-dotenv could not parse statement starting at line 18
Python-dotenv could not parse statement starting at line 24
Python-dotenv could not parse statement starting at line 27
Python-dotenv could not parse statement starting at line 28
Python-dotenv could not parse statement starting at line 29
Python-dotenv could not parse statement starting at line 30


True

In [8]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser


class Sentence(BaseModel):
    """Standard sentence """
    sentence: str = Field(..., description="The sentence")
    category: str = Field(..., description="The category of the sentence")
    

class Sentences(BaseModel):
    """Sentence structure correct and classification."""
    sentences: list[Sentence] = Field(..., description="List of sentences")
    
parser = PydanticOutputParser(pydantic_object=Sentences)

In [9]:
from langchain_core.prompts import ChatPromptTemplate

template_zero_shot = ChatPromptTemplate.from_messages(
    [
        ("system", """
You are a language expert in Japanese and English.
Your task is to process words in pre-existing sentences that have incorrect segmentation of word types, especially nouns that are not properly separated from other word types by spaces.
You will be provided with the nouns and the instructions needed to carry out the task.
Answer the user query. Wrap the output in `json` tags\n{format_instructions}"""),
        ("user", """
        あなたは優秀なSEMコンサルタントです。
        以下の 2 つのアクションを実行しください。
        ・アクション1：
        KWを日本語として適切な単語の品詞単位で区切ってください。
        ただし、次の条件を適用して書き直してください。
        ①「ライフネット 生命」は「ライフネット生命」で1つの名詞と判断
        ②「保険」は種別を含めて1つの名詞と判断
        例）「死亡」と「保険」を分けず、「死亡保険」で1つの単語と判断
        例）種別はがん(ガン、癌)、死亡、医療、入院、女性、定期、就業不能、認知症など
        ③処理されたクエリはスペースで区切られるようにしてください
        ・アクション2：
        次の手順で指定したKWをカテゴリに分類してください。
        ①下記優先度の高いカテゴリから順番に分類
        -競合他社
        例)ソニー、楽天、オリックス、メットライフ、SBI、アクサ、三井住友、アフラックなど
        -がん保険
        -死亡保険
        -就業不能保険
        -医療保険レディース
        -医療保険
        -ライフネット生命
        ②①に分類できなかったものは下記に分類してください。
        -「生命保険」
    
        Input sentences:
        {input}
        """),
    ]
).partial(format_instructions=parser.get_format_instructions())


In [10]:
text = """aflac 終身 保険
ja ポイント
ja 三重 中央 会
がん 保険 定期 終身
がん 保険 潰瘍 性 大腸 炎
がんかかる 費用
ゆとりある 生活費
アイコープ
イオンカードがん 保険 評判
ライフネット 生命 ev
介護 保険 値上げ
仙台 市 青葉 区 心療 内科
保険はこの5つから選びなさい
保険 控除
保険 適用 外の病気
個人 損害 賠償 保険
借金 返済 保険 解約
傷病 手当 いつからもらえる
入院 決まってから保険
医療 保険 30歳 女性"""
text = text.split("\n")
sentences: str = []
for i in text:
    sentences.append("'"+i+"'")


In [11]:
template_zero_shot.format_prompt(input=sentences).to_string()

'System: \nYou are a language expert in Japanese and English.\nYour task is to process words in pre-existing sentences that have incorrect segmentation of word types, especially nouns that are not properly separated from other word types by spaces.\nYou will be provided with the nouns and the instructions needed to carry out the task.\nAnswer the user query. Wrap the output in `json` tags\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "Sentence structure correct and classification.", "properties": {"sentences": {"title": "Sentences", "description": "List of sentence

In [21]:
from langchain_community.chat_models.azure_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name=os.getenv("OPENAI_API_LLM_DEPLOY_NAME_4_8k"),
    openai_api_type=os.getenv("OPENAI_API_TYPE"),
    openai_api_key=os.getenv("OPENAI_API_KEY_4"),
    azure_endpoint=os.getenv("OPENAI_API_BASE_URL"),
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    temperature=0.,
    # request_timeout=600,
    streaming=True,
)
model = template_zero_shot | llm | parser


In [22]:
import time
from langchain_community.callbacks import get_openai_callback

start = time.time()
try:
    with get_openai_callback() as cb:
        results = model.invoke({"input": sentences})
        token = cb
except Exception as error:
    print(error)
end = time.time()
print("total_time", end-start)


total_time 36.40451097488403


In [23]:
results

Sentences(sentences=[Sentence(sentence="'aflac 終身保険'", category='競合他社'), Sentence(sentence="'ja ポイント'", category='生命保険'), Sentence(sentence="'ja 三重 中央 会'", category='生命保険'), Sentence(sentence="'がん保険 定期終身'", category='がん保険'), Sentence(sentence="'がん保険 潰瘍性大腸炎'", category='がん保険'), Sentence(sentence="'がんかかる 費用'", category='がん保険'), Sentence(sentence="'ゆとりある 生活費'", category='生命保険'), Sentence(sentence="'アイコープ'", category='生命保険'), Sentence(sentence="'イオンカードがん保険 評判'", category='がん保険'), Sentence(sentence="'ライフネット生命 ev'", category='ライフネット生命'), Sentence(sentence="'介護保険 値上げ'", category='生命保険'), Sentence(sentence="'仙台 市 青葉 区 心療 内科'", category='生命保険'), Sentence(sentence="'保険はこの5つから選びなさい'", category='生命保険'), Sentence(sentence="'保険 控除'", category='生命保険'), Sentence(sentence="'保険 適用 外の病気'", category='生命保険'), Sentence(sentence="'個人 損害賠償保険'", category='生命保険'), Sentence(sentence="'借金 返済保険 解約'", category='生命保険'), Sentence(sentence="'傷病 手当 いつからもらえる'", category='生命保険'), Sentence(sentence="'入院決まってから保険'", categor

In [25]:
len(results.sentences)

20